Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Azure Machine Learning Pipeline with AutoMLStep 
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [8]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quickstart-ws-234016
quick-starts-ws-234016
ukwest
b74a5367-8189-4c03-b954-e3a41b64e370


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [9]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-bankmarketing-Project'
project_folder = './Project2/pipeline-bankmarketing-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bankmarketing-Project,quickstart-ws-234016,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [10]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-demo-3"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_E8s_v3',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [11]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bankmarketing Dataset"
description_text = "Bankmarketing dataset, regarding a campaign"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [12]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


#### Getting Registered Datasets

In [13]:
datasets_all = Dataset.get_all(ws)
print(datasets_all)


{ 'Bankmarketing Dataset': DatasetRegistration(id='d05365e6-92ae-472c-9f63-8f8b4a69c707', name='Bankmarketing Dataset', version=1, description='Bankmarketing dataset, regarding a campaign', tags={}),
  'Bike-sharing-Dataset': DatasetRegistration(id='aacff08c-b0ae-45ee-89ad-64fc1802e153', name='Bike-sharing-Dataset', version=1, description='', tags={}),
  'Bikesharing Dataset': DatasetRegistration(id='751d52b0-91b9-4a22-b4bd-546b7ec264c7', name='Bikesharing Dataset', version=1, description='Bike Sharing DataSet for Udacity Course 2', tags={})}


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [9]:
project_folder

'./Project2/pipeline-bike-project'

In [14]:
automl_settings = {
    "experiment_timeout_minutes": 60, #1 hour--> this value is in minute
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",  
                             path = project_folder,
                             enable_early_stopping= True,
                             #featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [15]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [16]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [17]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [18]:
from azureml.widgets import RunDetails

pipeline_run = experiment.submit(pipeline)
RunDetails(pipeline_run).show()

Created step automl_module [2bfec319][4975a9ea-0ad0-4461-9e9a-b47724e7b98d], (This step will run and generate new outputs)
Submitted PipelineRun edd6e86b-62ca-4d41-93ca-fa676995572b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/edd6e86b-62ca-4d41-93ca-fa676995572b?wsid=/subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourcegroups/quick-starts-ws-234016/workspaces/quickstart-ws-234016&tid=b33be5d6-5072-448f-bad3-d8b66cf09736


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [21]:
pipeline_run.wait_for_completion()

PipelineRunId: edd6e86b-62ca-4d41-93ca-fa676995572b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/edd6e86b-62ca-4d41-93ca-fa676995572b?wsid=/subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourcegroups/quick-starts-ws-234016/workspaces/quickstart-ws-234016&tid=b33be5d6-5072-448f-bad3-d8b66cf09736

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': 'edd6e86b-62ca-4d41-93ca-fa676995572b', 'status': 'Completed', 'startTimeUtc': '2023-06-06T11:34:48.811503Z', 'endTimeUtc': '2023-06-06T11:59:03.755291Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2023-06-06T11:34:49.1212039+00:00","EndTime":"2023-06-06T11:59:03.6197418+00:00","Status":"F

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [22]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/5ca5bdc6-0210-4234-afe8-94ff95f7f708/metrics_data, 1 files out of an estimated total of 1


In [23]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,5ca5bdc6-0210-4234-afe8-94ff95f7f708_7,5ca5bdc6-0210-4234-afe8-94ff95f7f708_5,5ca5bdc6-0210-4234-afe8-94ff95f7f708_16,5ca5bdc6-0210-4234-afe8-94ff95f7f708_21,5ca5bdc6-0210-4234-afe8-94ff95f7f708_13,5ca5bdc6-0210-4234-afe8-94ff95f7f708_18,5ca5bdc6-0210-4234-afe8-94ff95f7f708_26,5ca5bdc6-0210-4234-afe8-94ff95f7f708_9,5ca5bdc6-0210-4234-afe8-94ff95f7f708_30,5ca5bdc6-0210-4234-afe8-94ff95f7f708_20,...,5ca5bdc6-0210-4234-afe8-94ff95f7f708_3,5ca5bdc6-0210-4234-afe8-94ff95f7f708_4,5ca5bdc6-0210-4234-afe8-94ff95f7f708_6,5ca5bdc6-0210-4234-afe8-94ff95f7f708_10,5ca5bdc6-0210-4234-afe8-94ff95f7f708_11,5ca5bdc6-0210-4234-afe8-94ff95f7f708_12,5ca5bdc6-0210-4234-afe8-94ff95f7f708_29,5ca5bdc6-0210-4234-afe8-94ff95f7f708_31,5ca5bdc6-0210-4234-afe8-94ff95f7f708_27,5ca5bdc6-0210-4234-afe8-94ff95f7f708_33
recall_score_weighted,[0.9082245827010622],[0.8884370257966616],[0.9088922610015174],[0.9143247344461305],[0.8644613050075872],[0.9139605462822459],[0.8891654021244311],[0.9128376327769347],[0.8879514415781486],[0.8310773899848256],...,[0.9147192716236724],[0.9133535660091047],[0.9088012139605463],[0.9051289833080425],[0.9128679817905917],[0.9082852807283764],[0.9161153262518967],[0.9047344461305006],[0.9100455235204856],[0.9098330804248862]
recall_score_micro,[0.9082245827010622],[0.8884370257966616],[0.9088922610015174],[0.9143247344461305],[0.8644613050075872],[0.9139605462822459],[0.8891654021244311],[0.9128376327769347],[0.8879514415781486],[0.8310773899848256],...,[0.9147192716236724],[0.9133535660091047],[0.9088012139605463],[0.9051289833080425],[0.9128679817905917],[0.9082852807283764],[0.9161153262518967],[0.9047344461305006],[0.9100455235204856],[0.9098330804248862]
precision_score_weighted,[0.8960725075228633],[0.9008925765857139],[0.8968430956960398],[0.9055085288933681],[0.9200056329673243],[0.9052418720433252],[0.836147259025297],[0.9046491408814699],[0.7884613418500925],[0.9230642478221476],...,[0.9092879019492065],[0.9031451158026359],[0.8990694649900496],[0.8915127367505843],[0.9033571392265077],[0.8965731616057016],[0.9101774146331202],[0.9002305620426541],[0.9055667699359725],[0.9031879525330349]
f1_score_micro,[0.9082245827010622],[0.8884370257966616],[0.9088922610015174],[0.9143247344461305],[0.8644613050075872],[0.9139605462822459],[0.8891654021244308],[0.9128376327769347],[0.8879514415781486],[0.8310773899848254],...,[0.9147192716236722],[0.9133535660091047],[0.9088012139605463],[0.9051289833080425],[0.9128679817905917],[0.9082852807283764],[0.9161153262518967],[0.9047344461305006],[0.9100455235204856],[0.9098330804248862]
balanced_accuracy,[0.6839542824646604],[0.5021583592001966],[0.6836825667699283],[0.719415413306931],[0.8612755873918436],[0.7203334810591676],[0.5101080608259135],[0.7168854498146927],[0.5],[0.8760223524093254],...,[0.7498730750443867],[0.7006278002103492],[0.6980655237068734],[0.6166095647981475],[0.7049327677329552],[0.6884824292070767],[0.7484572934270534],[0.735689630191602],[0.7472496392007062],[0.7321505136951555]
AUC_macro,[0.930522182023228],[0.9411263132802157],[0.9280301759890343],[0.9472245502007045],[0.9329329064195605],[0.9461543597174187],[0.9216613553467068],[0.9438330535591982],[0.8642139000633506],[0.929943463742475],...,[0.9375209286331894],[0.9466026066703351],[0.932624203989719],[0.9348671167038767],[0.9436041635470518],[0.9310793886161493],[0.9439769690460811],[0.9314471987117866],[0.94125010265975],[0.937926400617236]
AUC_micro,[0.9739737174778542],[0.975726472030782],[0.9731178660820989],[0.9805412578491806],[0.9256940920740258],[0.9802487721083815],[0.9718606800665928],[0.9796448198286363],[0.9604255539616056],[0.9220131389584164],...,[0.9785452069052065],[0.9801680386662092],[0.976414376866591],[0.9764921099472461],[0.979568647949139],[0.973838892330081],[0.9800214124034898],[0.9749016604456561],[0.9783353059424659],[0.9780292207119354]
matthews_correlation,[0.4603385030040973],[0.060284225829192004],[0.46289784817494384],[0.5145539995232143],[0.5550666212308545],[0.5140519046

### Retrieve the Best Model

In [40]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/a49e82b7-6fde-4a24-b6c7-7e9d3f58841a/model_data, 1 files out of an estimated total of 1


In [41]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('33', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alp

In [19]:
best_model.steps

[('timeseriestransformer',
  TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(blocked_transformers=None, column_purposes=None, dataset_language=None, prediction_transform_type=None, transformer_params=None), force_time_index_features=None, freq='D', grain_column_names=['_automl_dummy_grain_col'], group=None, lookback_features_removed=False, max_horizon=1, origin_time_colname='origin', pipeline=Pipeline(memory=None, steps=[('unique_target_grain_dropper', UniqueTargetGrainDropper(cv_step_size=1, max_horizon=1, n_cross_validations=5, target_lags=[0], target_rolling_window_size=0)), ('make_numeric_na_dummies', MissingDummiesTransformer(numerical_columns=['instant', 'season', 'yr', 'mnth', 'weekday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered'])), ('impute_na_numeric_datetime', TimeSeriesImputer(end=None, freq='D', impute_by_horizon=False, input_column=['instant', 'season', 'yr', 'mnth', 'weekday', 'we

### Enable Application Insight 

In [24]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

ws=Workspace.from_config()

# get the service name from end-point section
name= "bankmarketing-project-demo"

#load existing web service
service = Webservice(name=name,workspace=ws)

service.update(enable_app_insights=True)




In [27]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "bankmarketing-project-demo"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)


2023-06-06T12:13:11,861712153+00:00 - rsyslog/run 
2023-06-06T12:13:11,876291252+00:00 - gunicorn/run 
2023-06-06T12:13:11,889943952+00:00 | gunicorn/run | 
2023-06-06T12:13:11,893501252+00:00 | gunicorn/run | ###############################################
2023-06-06T12:13:11,900450451+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-06-06T12:13:11,909691351+00:00 | gunicorn/run | ###############################################
2023-06-06T12:13:11,926942650+00:00 - nginx/run 
2023-06-06T12:13:11,927022050+00:00 | gunicorn/run | 
2023-06-06T12:13:11,939124449+00:00 | gunicorn/run | 
2023-06-06T12:13:11,954056249+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230412.v4
2023-06-06T12:13:11,959822848+00:00 | gunicorn/run | 
2023-06-06T12:13:11,965971348+00:00 | gunicorn/run | 
2023-06-06T12:13:11,980747547+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [28]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quickstart-ws-234016
quick-starts-ws-234016
ukwest
b74a5367-8189-4c03-b954-e3a41b64e370


In [29]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'ml-bankmarketing-Project'
project_folder = './pipeline-project-bankmarketing'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-bankmarketing-Project,quickstart-ws-234016,Link to Azure Machine Learning studio,Link to Documentation


### Get the run id:

In [30]:
from azureml.core import Experiment

# get the experiment
experiment = Experiment(workspace=ws, name='ml-bankmarketing-Project')

# get a list of runs for the experiment
runs = list(experiment.get_runs())

# get the ID of the first run
run_id = runs[0].id

print(run_id)


edd6e86b-62ca-4d41-93ca-fa676995572b


In [31]:
from azureml.pipeline.core import PipelineRun

run_id = "edd6e86b-62ca-4d41-93ca-fa676995572b"
pipeline_run = PipelineRun(experiment, run_id)

In [32]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bank Marketing", description="Bank marketing AutoML pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bank Marketing,cf909b53-2504-4958-a0bf-f1b82fc84861,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [33]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [34]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bankmarketing-rest-endpoint"}
                        )

In [35]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  d4fe516b-8bd8-4249-8c22-528002534105


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [36]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bankmarketing-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …